### The data frame contains daily sales quantities of 3 different stores. We first create a period of 10 days using the date_range function of pandas.

In [62]:
import pandas as pd
import numpy as np

In [63]:
# returns a fixed frequency datetime index
days=pd.date_range("2021-01-01",periods=10,freq='D')

In [64]:
days

DatetimeIndex(['2021-01-01', '2021-01-02', '2021-01-03', '2021-01-04',
               '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08',
               '2021-01-09', '2021-01-10'],
              dtype='datetime64[ns]', freq='D')

In [65]:
A=pd.DataFrame({"date":days,"store":"A","sales":np.random.randint(100,200,size=10)})
B=pd.DataFrame({"date":days,"store":"B","sales":np.random.randint(100,200,size=10)})
C=pd.DataFrame({"date":days,"store":"C","sales":np.random.randint(100,200,size=10)})

In [66]:
df=pd.concat([A,B,C]).sort_values("date")

### Let’s suppose we need to find the store that has the highest sales quantity for each day. This task can be done using the groupby and max functions.

In [67]:
df=df.reset_index(drop=True)

In [68]:
df.head()

,date,store,sales
0,2021-01-01,A,172
1,2021-01-01,C,164
2,2021-01-01,B,122
3,2021-01-02,A,117
4,2021-01-02,C,160


In [120]:
s=df.groupby(by=["date"])["sales"].max()
df_agg=pd.DataFrame({"date":s.index,"max_sales":pd.Series(s.values)})

In [121]:
df_agg

,date,max_sales
0,2021-01-01,172
1,2021-01-02,191
2,2021-01-03,157
3,2021-01-04,180
4,2021-01-05,177
5,2021-01-06,196
6,2021-01-07,142
7,2021-01-08,186
8,2021-01-09,173
9,2021-01-10,137


In [142]:
df_final=pd.DataFrame(columns=["date","store","sales"])

In [143]:
for i in range(10):
    df_final=pd.concat([df_final,df.loc[(df["date"]==df_agg["date"][i]) & (df["sales"]==df_agg["max_sales"][i])]],axis=0)

### This is our transformed table which contains the store with the max sales on a given date 

In [145]:
df_final

,date,store,sales
0,2021-01-01,A,172
5,2021-01-02,B,191
7,2021-01-03,C,157
10,2021-01-04,C,180
14,2021-01-05,A,177
16,2021-01-06,A,196
19,2021-01-07,A,142
21,2021-01-08,B,186
25,2021-01-09,B,173
28,2021-01-10,A,137


### We can see that value counts function enables us to find that out of 10 days , store A had max sales on 5 days , but if we have to find store A's all possible positions and their corresponding frequencies , rank function comes to our aid and greatly simplifies our efforts , let's explore it 

In [146]:
df_final["store"].value_counts()

A    5
B    3
C    2
Name: store, dtype: int64

In [149]:
df["rank"]=df.groupby("date")["sales"].rank(ascending=False).astype(int)

In [152]:
df.head(6)

,date,store,sales,rank
0,2021-01-01,A,172,1
1,2021-01-01,C,164,2
2,2021-01-01,B,122,3
3,2021-01-02,A,117,3
4,2021-01-02,C,160,2
5,2021-01-02,B,191,1


In [155]:
df.groupby(["store","rank"]).count()["sales"]

store  rank
A      1       5
       2       3
       3       2
B      1       3
       3       7
C      1       2
       2       7
       3       1
Name: sales, dtype: int64

### Store A has the highest number of first ranks. Store B is usually the second store and C has kind of a uniform distribution of ranks but in this case we cannot see the name of the aggregated column and hence it wouldn't be of much sense to someone who has not gone through the code , so let's use name aggregation

In [157]:
df.groupby(["store","rank"]).agg(rank_count=("rank","count"))

rank_count
store rank            
A     1              5
      2              3
      3              2
B     1              3
      3              7
C     1              2
      2              7
      3              1